# Analyzing Stock Market Data to Build a Portfolio

## Project Goal

### Analyze share price data from various listings to answer the following questions:

1. What kind of gains do we expect from the various listings? <br>
2. How do the price gains of the different listings correlate to each other? <br>
3. What is the estimated listing price/stock at a future date (prediction)? <br>
4. What kind of volatility do the listings have? <br>
5. How can we use our analysis to form a balanced investment portfolio? <br>

### Overall Methodology:

1. Import and organize data for the various listings <br>
2. Make observations about the data <br>
3. Use these observations to answer the questions <br>

## References

Share price information obtained from: [Yahoo Finance](https://finance.yahoo.com/) <br>
Link to Blogpost: [An Investing Newbie's Tryst with Data Science](https://medium.com/@kgraghav/an-investing-newbies-tryst-with-data-science-241737102a6a) <br>

## Inputs

### Below are the inputs to perform the analysis

In [1]:
s_list='AAPL FB NVDA TSLA FCAU F AAL UAL INO MVIS'  # List of "listings" to be analyzed
total_investment=10000  # Total investment value in USD
sample_interval ='1d'  # Time interval to fetch data at
start='2019-09-10'  # Start date of interest
end='2020-09-09'  # End period of interest
resample_interval='3d'  # Upsampling/Downsampling interval (determined through trial and error)
datetime_query=['09/21/2020','10/21/2021'] # Datetime query values (for prediction)

<b>For this analysis, we look at the following listings (from different industries): </b> <br>
<table style="width:100%">
  <tr><tr align="Center">
    <th>Listed Name</th>
    <th>Company Name</th>
    <th>Industry</th>
  </tr>
  <tr>
    <tr align="Center">
    <td>AAPL</td>
    <td>Apple</td>
    <td>Technology</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>FB</td>
    <td>Facebook</td>
    <td>Technology</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>NVDA</td>
    <td>NVDIA</td>
    <td>Technology</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>TSLA</td>
    <td>Tesla</td>
    <td>Auto (Electric)</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>FCAU</td>
    <td>FCA</td>
    <td>Auto</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>F</td>
    <td>Ford</td>
    <td>Auto</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>AAL</td>
    <td>American Airlines</td>
    <td>Airline</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>UAL</td>
    <td>United Airlines</td>
    <td>Airline</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>INO</td>
    <td>Inovio</td>
    <td>Medical</td>
  </tr>
  <tr>
    <tr align="Center">
    <td>MVIS</td>
    <td>Microvision</td>
    <td>Medical</td>
  </tr>
</table>

## Import Libraries

In [ ]:
#Update python standard libraries install
!pip install --upgrade pip

# library to for array handling
import numpy as np 

# library for dataframes
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Linear Model
import sklearn as sk

# Math Module
import math

# Matplotlib and associated plotting modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# Machine-Learning and analysis modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Module to handle XML and HTML 
! pip install lxml;

#Finance information import (Link in "Yahoo Finance" Markdown Cell in the "References" Section)
!pip install yfinance;
import yfinance as yf;

print('Libraries imported.')

Requirement already up-to-date: pip in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (20.2.3)


## Import Data into DataFrame

### Create basic dataframe "df_info" containing all the available stocks information

In [ ]:
# Create list of stock names as specified by the User
s_list=s_list.split(' ')

In [ ]:
# Initialize Basic Dataframe as empty
df_info=pd.DataFrame() 

# Add Stocks information for List of stock names
for item in s_list:
    data=yf.Ticker(item)
    df_data=data.history(period=sample_interval,start=start ,end=end )
    df_data=df_data.reset_index()
    df_data['Name']=pd.DataFrame([item for i in range(0,len(df_data))])
    df_info=pd.concat([df_info,df_data])

# Display basic information
df_info.head()

In [ ]:
# Size of Data:
print('Size of the full dataset is: {} by {}'.format(df_info.shape[0],df_info.shape[1]))

## Exploratory Pre-Processing Analysis

### Create Dataframe (df) of Opening Price vs. date, which will be the data looked at in this analysis  

In [ ]:
# Create new Dataframe of Opening Stock Prices
df=pd.DataFrame();
for item in s_list:
    df[['Date',item]]=df_info.loc[df_info['Name']==item,['Date','Open']]
df=df.set_index('Date')
df_price=df
# Display initial few contents of the Dataset
df.tail(10)

### Display basic information about this dataset

In [ ]:
# Display basic statistics of the opening prices
# Darker cells denote higher values
df.describe().style.background_gradient(axis=1)

<b> A few notes and observations: </b> </br>
1. All values in the "count" row are the same, implying there are no bad values (such as NaNs) <br>
2. NVDIA and TESLA have some of the highest price fluctuations in absolute value based on their std. <br>
3. The listings looked at in the tech. industry have some of the highest share prices, followed by the airline industry. The Auto industry is generally lower with the exception of Tesla, whose share price is on par with the tech. industry (what's different here?)<br>
4. NVDIA and TESLA have seen some of the highest rises in absolute prices over the period <br>
5. Inovio (medical industry) has the highest rise in percentage of initial price <br>

### Create basic Time plot to look at pertinent information (Opening price)

In [ ]:
# Plot Open price vs. Time
ax= plt.subplot()
for item in df_price.columns:
    plt.plot(df_price[item])

ax.legend(df_price.columns)
plt.ylabel('Stock Price [USD]')
ax.grid(True)

<b>Again a few things to note: </b><br>
1. Tech industry stocks seem to follow a similar trend w.r.t. time <br>
2. Tech industry stocks are seen to rise steadily. <br>
3. All listings had a decline around the 2020-03 to 2020-04 timeframe. <br>
4. Tech industry stocks rebounded back much better than the other industry listings after 2020-04. Tesla is the exception which more closely forllows the trends of the Tech. industry <br>

## What kind of gains do we see with the various listings?

### Normalize the opening price to determine % Gain over the period of interest

Normalize w.r.t. start price (df_start) <br> 
Normalization used: (price-start_price)/start_price*100 to obtain df_norm

In [ ]:
# Normalization Function
def df_normalize(df):
    df_strt=df.iloc[0,:]
    df_norm=(df.iloc[:,:]-df_strt)/df_strt*100
    df_strt=pd.DataFrame(df_strt).transpose()
    df_strt=df_strt.set_axis(['start_price'])
    return [df_norm,df_strt]

# Store Normalized values in DataFrame and display results                          
[df_norm,df_start]=df_normalize(df)
df_norm.head()

In [ ]:
# Size of Data:
print('Size of the data is: {} by {}'.format(df_norm.shape[0],df_norm.shape[1]))

In [ ]:
# Display basic statistics of the normalized prices
# Darker cells denote higher values
df_norm.describe().style.background_gradient(axis=1)

### Plot trends in Normalized values

Time Plot

In [ ]:
# Display % Growth over time
ax= plt.subplot()
for item in df_norm.columns:
    plt.plot(df_norm[item])

ax.legend(df.columns)
plt.ylabel('Growth %')
ax.grid(True)

## How do the price gains of the listings correlate to each other?

Correlation Matrix

In [ ]:
sns.heatmap(df_norm.corr(),annot=True);

In [ ]:
ax= plt.subplot()
for item in df_norm.columns:
    plt.scatter(df_norm.iloc[:,0],df_norm[item])

ax.legend(df_norm.columns)
plt.xlabel('AAPL Growth %')
plt.ylabel('Growth %')
ax.grid(True)

<b>Observations:</b> <br>
1. Best matches between various companies in the tech industry (and Tesla) <br>
2. Worst matches between the tech industry and airline industry <br>
3. Medical industry has intermediate match with the tech industry and bad match with the airline and auto industry<br>
4. Auto and airline industries correlate well with each other <br>

### Some general observations and next steps

1. Time Data is too transient to determine good (suitably longer term) trends of rise and fall of prices <br>
2. To alleviate this, the data is downsampled so that the transients are smoothed out and better estimates can be made for rise and fall trends

## General approach to answer the remaining questions:

1. Downsample the data to smoothen it and obtain longer term trends in rise and fall of prices <br>
2. Use the smoothened data to obtain price rise and price fall information <br>
3. Use the standard deviation of the price rise/fall values to determine the price bounds  <br>
4. Use the ratio of price bounds and the change in price, to estimate "volatility" <br>
5. Create a linear regression fit object for the historical data over the period of analysis <br>
6. Estimate the "reward" or slope of price change from the fit object <br>
7. Plot the historical data along with the linear fit and price bounds to see how well the historical data falls within these margins <br>
8. Estimate the "returns_ratio" as the ratio between the "reward" and "volatility" for each listing <br>
9. The amount to invest in each listing is the weighted average of the "returns_ratio" multiplied by the total investment capital <br>

### Downsample the data to filter out high frequency changes and determine pertinent trends

<b> Downsampled normalized data  "df_norm_resampled"</b>

In [ ]:
# Downsample with cubic interpolation
df_norm_resampled=df_norm.resample(resample_interval).interpolate(method='cubic')
df_norm_resampled.head()

<b> Downsampled normalized data statistics "df_norm_resampled" </b>

In [ ]:
# Basic downsampled statistics of normalized data
df_norm_resampled.describe().style.background_gradient(axis=1).format("{:.0f}")

<b> Downsampled price data </b>

In [ ]:
# Downsample with cubic interpolation
df_resampled=df.resample(resample_interval).interpolate(method='cubic')
df_resampled.head()

<b> Downsampled price data statistics </b>

In [ ]:
# Basic downsampled statistics of normalized data
df_resampled.describe().style.background_gradient(axis=1).format("{:.0f}")

<b>Time plot of filtered price data</b>

In [ ]:
ax= plt.subplot()
for item in df_resampled.columns:
    plt.plot(df_resampled.index,df_resampled[item])

ax.legend(df_resampled.columns)
plt.ylabel('Price USD')
ax.grid(True)

We are able to sufficiently capture trends now after the filtering.<br>

### Calculate Time, percent and absolute rise and fall of Growth 

1. "df_norm_resampled_rise_fall" is the dataframe of the normalized rise and fall prices <br>
2. "df_norm_resampled_rise_fall_time" is the dataframe of the rise and fall times of the normalized changes <br>
3. "df_norm_resampled_rise_fall_prices" is the dataframe of the rise and fall prices, calculated from the normalized values and the start value <br>

In [ ]:
#Function to calculate rise and fall: rise_fall_stat(df,item)
# "item" is a list of columns of the price dataframe
def rise_fall_stat(df,item):
    #Initialize rise and fall arrays
    rise=np.array([])        
    fall=np.array([])
    rise_t=np.array([])        
    fall_t=np.array([])
    # if df[i]-df[i-1] is positive, 
    # add that to rise and add the time delta to rise_time
    # else if negative, add that (absolute) to fall, and add the time delta to fall_time
    # else move to next point
    i=1
    while i<len(df)-1:
        rise_val=0
        fall_val=0
        rise_time=0
        fall_time=0
        try:
            while df.iloc[i]>df.iloc[i-1] and i<len(df)-1:
                rise_val=rise_val+(df.iloc[i]-df.iloc[i-1])
                rise_time=rise_time+((df.index[i]-df.index[i-1]).value)
                i=i+1
            rise=np.append(rise,rise_val)
            rise_t=np.append(rise_t,rise_time)
        except:
            print('{} error at: {}, {}'.format(e,item,i))
        try:
            while df.iloc[i]<df.iloc[i-1] and i<len(df)-1:
                fall_val=fall_val+(df.iloc[i-1]-df.iloc[i])
                fall_time=fall_time+((df.index[i]-df.index[i-1]).value)
                i=i+1
            fall=np.append(fall,fall_val)
            fall_t=np.append(fall_t,fall_time)
        except:
            print('{} error at: {}, {}'.format(e,item,i))
        i=i+1
    # Estimate statistics for rise and fall arrays and create a dataframe for it
    df_rise_fall=pd.DataFrame({'%_rise_mean': [np.mean(rise)],'%_fall_mean': [np.mean(fall)],
                               '%_rise_min': [np.min(rise)],'%_fall_min': [np.min(fall)],
                               '%_rise_max': [np.max(rise)],'%_fall_max': [np.max(fall)],
                               '%_rise_std': [np.std(rise)],'%_fall_std': [np.std(fall)]}).transpose()
    df_rise_fall=df_rise_fall.rename(columns={0:item})
    # Estimate statistics for rise and fall arrays and create a dataframe for it
    df_rise_fall_time=pd.DataFrame({'time_rise_mean': pd.to_timedelta([np.mean(rise_t)]).days,
                                    'time_fall_mean': pd.to_timedelta([np.mean(fall_t)]).days,
                               'time_rise_min': pd.to_timedelta([np.min(rise_t)]).days,
                                    'time_fall_min': pd.to_timedelta([np.min(fall_t)]).days,
                               'time_rise_max': pd.to_timedelta([np.max(rise_t)]).days,
                                    'time_fall_max': pd.to_timedelta([np.max(fall_t)]).days,
                               'time_rise_std': pd.to_timedelta([np.std(rise_t)]).days,
                                    'time_fall_std': pd.to_timedelta([np.std(fall_t)]).days}).transpose()
    df_rise_fall_time=df_rise_fall_time.rename(columns={0:item})
    # Return rise, fall dataframes for price and time
    return [df_rise_fall,df_rise_fall_time]

In [ ]:
# Calculate rise and fall in gain % (w.r.t. initial price) for each listing using rise_fall_stat(...)
df_norm_resampled_rise_fall=pd.DataFrame()
df_norm_resampled_rise_fall_time=pd.DataFrame()
# for each listing, obtain the rise and fall prices and times:
for name in df_norm_resampled.columns:
    df_norm_resampled_rise_fall=pd.concat([df_norm_resampled_rise_fall,rise_fall_stat(df_norm_resampled[name],name)[0]],axis=1)
    df_norm_resampled_rise_fall_time=pd.concat([df_norm_resampled_rise_fall_time,rise_fall_stat(df_norm_resampled[name],name)[1]],axis=1)

In [ ]:
# Display values for each listing. Darker cells denote larger values
df_norm_resampled_rise_fall.style.background_gradient(axis=1).format("{:.0f}")

In [ ]:
# Display values for each listing. Darker cells denote larger values
df_norm_resampled_rise_fall_time.style.background_gradient(axis=1)

Convert rise and fall from % to price (using price change= percent change/100*start price) and display.<br>

In [ ]:
print('df_start={}'.format(df_start))
df_norm_resampled_rise_fall_prices=pd.DataFrame(df_norm_resampled_rise_fall.values/100*df_start.values)
# set appropriate index axis:
df_norm_resampled_rise_fall_prices=df_norm_resampled_rise_fall_prices.set_axis(
    [index.replace('%','price') for index in df_norm_resampled_rise_fall.index],
                                                 axis=0).set_axis(df_norm_resampled_rise_fall.columns,axis=1)
df_norm_resampled_rise_fall_prices.style.background_gradient(axis=1).format("{:.0f}")

### Create a linear regression fit for the data with 3 std. of the maximum of rise and fall to determine the boundaries

<b> Linear Regression object, score and price boundaries for each listing </b>

In [ ]:
def lin_obj(df):
    df_lin=df.iloc[0:1,:]
    df_lin=df_lin.reindex(['fit_obj','score','price_bounds'])
    for item in df_lin.columns:
        price_bound=np.maximum(df_norm_resampled_rise_fall_prices.loc['price_rise_std',item],
                                  df_norm_resampled_rise_fall_prices.loc['price_fall_std',item])*3
        df_lin.loc['price_bounds',item]=price_bound
        
        # Implement train-test split thrice and fit the data to the average fit ...
        # ... (intercept and coeff.) of the three splits
        X=np.array(pd.to_numeric(df.index)).reshape(-1, 1)
        Y=(df[item].values).reshape(-1, 1)
        lin_list_coeff=[]
        lin_list_intercept=[]
        for i in range(0,2):
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)
            lin_obj=LinearRegression()
            lin_obj.fit(X_train,y_train)
            lin_list_coeff.append(lin_obj.coef_)
            lin_list_intercept.append(lin_obj.intercept_)
        linobj = LinearRegression()
        linobj.coef_=np.array(np.array(lin_list_coeff).mean()).reshape(-1)
        linobj.intercept_=np.array(np.array(lin_list_intercept).mean()).reshape(-1)
        score=linobj.score(X,Y)
        df_lin.loc['fit_obj',item]=linobj
        df_lin.loc['score',item]=score
        df_lin=df_lin.rename_axis(index='')
        
    return df_lin

In [ ]:
# Call lin_obj(...) to create the linear regression object dataframe
df_price_est_obj=lin_obj(df)
df_price_est_obj

### Estimate the price and bounds vs. date using the fit object

In [ ]:
# Function to estimate the price given the price dataframe, fit objects and datetime query
def price_est(df,df_price_est_obj,datetime_query):
    df_price_est=pd.DataFrame()
    df_price_est=df_price_est.rename_axis(index='Datetime')
    df_price_est['Datetime']=pd.to_datetime(datetime_query)
    df_price_est=df_price_est.set_index('Datetime')
    for item in df.columns:
        fit_obj=df_price_est_obj.loc['fit_obj',item]
        for datetime in df_price_est.index:
            price_est=fit_obj.predict(np.array(datetime.value).reshape(-1,1))
            df_price_est.loc[datetime,'{}'.format(item)]=price_est
            df_price_est.loc[datetime,'{}_low'.format(item)]=price_est-(df_price_est_obj.loc['price_bounds',item])
            df_price_est.loc[datetime,'{}_high'.format(item)]=price_est+(df_price_est_obj.loc['price_bounds',item])
    df_price_est=df_price_est.reindex(df_price_est.index.date)
    df_price_est=df_price_est
    return df_price_est


## What is the predicted price/stock?

In [ ]:
df_price_est=price_est(df,df_price_est_obj,datetime_query)
df_price_est.head().style.background_gradient(axis=1).format("{:.0f}")

## What kind of volatility do we see in the stock prices?

## How do we generate our investment portfolio?

### Estimate the price history along with the bounds and plot the history and estimates for each listing

In [ ]:
# Datetime history query values
datetime_query_start_end=pd.date_range(start=start,end=end)

In [ ]:
# Generate and display estimates (listing, listing_low, listing_high)
df_price_est_start_end=price_est(df,df_price_est_obj,datetime_query_start_end)
df_price_est_start_end.head().style.background_gradient(axis=1).format("{:.0f}")

In [ ]:
# Plot Open price vs. Time
num_plot=len(df.columns)
i=1
for item in df.columns:
    plt.figure()
    plt.plot(df[item])
    plt.plot(df_price_est_start_end[[item,item+'_low',item+'_high']])
    plt.legend([item,item+'_predicted',item+'_low',item+'_high'])
    plt.ylabel('Stock Price [USD]')
    plt.grid(True)
    #plt.ylim(0,700)
    plt.title(item)
    i=i+1

## Constructing the Portfolio

In order to balance the investments, we'd want to have the most benefit (reward) while minimizing volatility (risk).<br>
1. We can consider 'reward' to be the slope of the linear fit to share price (higher the slope, higher the gain).<br>
2. We can consider 'risk' to be the ratio between the price bounds and change in price over the duration of interest <br>
3. Hence we can compute a 'returns ratio' = 'reward'/'risk' for each listing; the higher this value, the more this investment is viable. <br>
4. Then we can compute the sum of the returns ratios for each listing and divide the returns ratio by the summed value, to determine the "weight" of each investment <br>
5. Finally we can multiply the "weight" by the total investment capital (total_investment) to obtain the recommended investment for each listing <br>    

### Use the Linear Regression object to construct the reward, volatility, returns ratio, weight and suggested investment values

In [ ]:
# Initialize the portfolio dataframe df_portfolio and update the index suitably
df_portfolio=df_price_est_obj
df_portfolio=df_portfolio.reindex(['reward','risk','returns_ratio','weight','suggested_investment'])
df_portfolio=df_portfolio.rename_axis(index='Parameters')

# Update "reward" using the slope of the line fit (multiplied by a suitable factor for display)
for item in (df_portfolio.columns):
    df_portfolio.loc['reward',item]=(df_price_est_obj.loc['fit_obj',item].coef_[0])*10**17
    
# Update the "volatility" as the ratio of the price bounds to the absolute net price change over the duration
for item in (df_portfolio.columns):
    df_portfolio.loc['risk',item]=(df_price_est_obj.loc['price_bounds',
                                                              item])/abs(df_price_est_start_end.loc[df_price_est_start_end.index[-1],
                                                                                                 item]-
                                         df_price_est_start_end.loc[df_price_est_start_end.index[0],
                                                                    item])
                                         
# Update the returns ratio as the ratio between the "reward" and "volatility"
for item in (df_portfolio.columns):
    df_portfolio.loc['returns_ratio',item]=(df_portfolio.loc['reward',item]/
                                            df_portfolio.loc['risk',item])
    
# Remove those listings with negative returns_ratio since those imply a falling stock
for item in df_portfolio.columns:
    if df_portfolio.loc['reward',item]<=0:
        df_portfolio.pop(item)
        
# Obtain the weighted average of the returns ratio for each listing
for item in (df_portfolio.columns):
    df_portfolio.loc['weight',item]=df_portfolio.loc['returns_ratio',item]/df_portfolio.loc['returns_ratio',:].sum()

# Multiply the weight of each listing by the total investment capital to obtain the recommended investment
for item in (df_portfolio.columns):
    df_portfolio.loc['suggested_investment',item]=df_portfolio.loc['weight',item]*total_investment

# Display the portfolio dataframe
df_portfolio.style.background_gradient(axis=1).format("{:.2f}")

### What kind of volatility do the listings have?

 Refer the "volatility" row, higher the value, greater the volatility

### How do we use the data to construct our portfolio?

Refer the dataframe "df_portfolio"